In [1]:
import torch
import torch.nn as nn

### MaxPool2d

In [9]:
x = torch.randn(2,5,4)
x

tensor([[[ 0.7325, -0.4693,  1.3016,  0.0429],
         [ 0.6522, -0.7112,  1.1784,  0.1337],
         [-1.5074,  0.6961, -0.9930,  0.0461],
         [-0.1465,  0.4056, -0.3168,  0.8870],
         [ 1.0842, -0.3936,  1.3609,  0.2288]],

        [[ 0.9562,  0.7429,  2.8072, -0.6907],
         [ 0.9524,  0.9143, -1.1879,  1.3792],
         [ 0.0925,  1.0983, -0.4352, -0.0272],
         [ 0.9869, -0.5733,  0.5801, -0.5800],
         [-0.1300,  0.7630, -1.0576, -0.6069]]])

In [10]:
pooling = nn.MaxPool2d(kernel_size=2,stride=1,padding=0)
pooling(x)

tensor([[[0.7325, 1.3016, 1.3016],
         [0.6961, 1.1784, 1.1784],
         [0.6961, 0.6961, 0.8870],
         [1.0842, 1.3609, 1.3609]],

        [[0.9562, 2.8072, 2.8072],
         [1.0983, 1.0983, 1.3792],
         [1.0983, 1.0983, 0.5801],
         [0.9869, 0.7630, 0.5801]]])

In [15]:
conv_33 = nn.Conv2d(in_channels=2, out_channels=3,kernel_size=3,stride=2)
conv_33(x)

tensor([[[-0.7692],
         [ 0.4855]],

        [[-1.2355],
         [ 0.3495]],

        [[ 0.7657],
         [-0.1098]]], grad_fn=<SqueezeBackward1>)

In [28]:
from torch.nn import functional as F

class Residual(nn.Module):  #@save
    """The Residual block of ResNet."""
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
Residual(3,3)

Residual(
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [36]:
class Residual_Block(nn.Module):
    def __init__(self,in_channels,num_channels,num_layers):
        super().__init__()
        self.first = Residual(in_channels,num_channels,use_1x1conv=True,strides=2)
        self.num_regulars = num_layers-1
        self.blocks = []
        for i in range(self.num_regulars):
            self.blocks.append(Residual(num_channels,num_channels))
    def forward(self,X):
        X = self.first(X)
        for blk in self.blocks:
            X = blk(X)
        return X


In [37]:
class ResNet34(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,64,7,2,padding=(3, 3))
        self.bn1 =  nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(3,2,1,1)
        self.layer1 = nn.Sequential(
            Residual(64,64),
            Residual(64,64),
            Residual(64,64)
        )
        self.layer2 = Residual_Block(64,128,4)
        self.layer3 = Residual_Block(128,256,6)
        self.layer4 = Residual_Block(256,512,4)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=512, out_features=1000, bias=True)
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x
ResNet34()

ResNet34(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,